In [26]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.pydantic_v1 import BaseModel, Field
import pickle
import os
from neo4j import GraphDatabase
from langchain_community.vectorstores import Neo4jVector

from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.prompts import ChatMessagePromptTemplate, PromptTemplate

from typing import Tuple, List, Optional

from langchain_ollama import ChatOllama

In [2]:
NEO4J_URI="neo4j+s://9269fc71.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="lOnpSSiAFQg0uWzA1EvYYBhK45_rWbxXb9SHRDwZRfk"

In [3]:
# Configure Neo4j and load embeddings
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [4]:
graph = Neo4jGraph()

In [5]:
graph_documents = Neo4jGraph(
    url=os.environ["NEO4J_URI"], username=os.environ["NEO4J_USERNAME"], password=os.environ["NEO4J_PASSWORD"]
)

In [6]:
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name, model_kwargs=model_kwargs)

/tmp/ipykernel_13485/995793401.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name, model_kwargs=model_kwargs)
/home/mouli/miniconda3/envs/gastro/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [9]:
vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [27]:
llm = ChatOllama(model='llama3.2', temperature=0)

In [10]:
# llm = OllamaLLM(model="llama3.2")

In [11]:
fusion_template = """
        Task: You are an assistant that generates multiple variations of a given question. 
        For each variation, maintain the original intent of the question, but change the phrasing, structure, 
        or tone to create a diverse set of queries.

Generate 5-7 variations that cover:

Synonym replacements while keeping the question concise.
Alternative structures, such as rephrasing into "why," "how," or "what" forms if relevant.
Casual and formal tones.
Slightly more specific or broader wording.
Examples:

Original Question: "How is a gastric ulcer diagnosed?"
Variations:
    "What tests can be used to detect stomach ulcers?"
    "Why are endoscopies commonly used in gastric ulcer diagnosis?"
    "How do doctors figure out if someone has a stomach ulcer?"
    "Can an upper endoscopy be used to diagnose all types of gastric ulcers, or are there other methods more effective for certain cases?"
    "What tests and procedures are typically involved in diagnosing gastric ulcers, including acid reflux disease?"
    "How can a patient determine if they have been diagnosed with a stomach ulcer based on symptoms, test results, or imaging studies?"
    "When considering the diagnosis of a gastric ulcer, what role does symptom severity play in determining the need for further testing or treatment?"


the final answer should be a python list: [
    "What tests can be used to detect stomach ulcers?",
    "Why are endoscopies commonly used in gastric ulcer diagnosis?",
    "How do doctors figure out if someone has a stomach ulcer?",
    "Can an upper endoscopy be used to diagnose all types of gastric ulcers, or are there other methods more effective for certain cases?",
    "What tests and procedures are typically involved in diagnosing gastric ulcers, including acid reflux disease?",
    "How can a patient determine if they have been diagnosed with a stomach ulcer based on symptoms, test results, or imaging studies?",
    "When considering the diagnosis of a gastric ulcer, what role does symptom severity play in determining the need for further testing or treatment?"
]

Now generate the list of variations for the given question
        """

In [28]:
prompt = ChatPromptTemplate(input_variables=['original_query'],
                            messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[],template=fusion_template)),
                            HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['original_query'], template='Generate multiple search queries related to: {question} \n OUTPUT (5 queries):'))])
     

In [29]:
original_query = 'How do NSAIDs induce gastric mucosal injury leading to ulcers?'

In [30]:
generate_queries = prompt | llm | StrOutputParser() | (lambda x: x.split("\n"))

In [41]:
generate_queries.invoke('How do NSAIDs induce gastric mucosal injury leading to ulcers?')

['Here are 5 variations of the search query:',
 '',
 '1. "What is the mechanism by which NSAIDs cause stomach lining damage and lead to ulcer formation?"',
 '2. "How do nonsteroidal anti-inflammatory drugs (NSAIDs) contribute to gastric mucosal injury and subsequent ulcers?"',
 '3. "Why does taking NSAIDs increase the risk of developing stomach ulcers, and what are the underlying biochemical pathways involved?"',
 '4. "Can chronic use of NSAIDs lead to long-term damage to the stomach lining, resulting in ulcer formation, and if so, how does this occur?"',
 '5. "What is the relationship between NSAID-induced gastric mucosal injury and the development of peptic ulcers, including the role of acid secretion and mucus production?"',
 '',
 'These variations maintain the original intent of the question while changing the phrasing, structure, or tone to create a diverse set of queries.']

In [32]:
# Define the schema without validators to avoid deepcopy issues
class Entities(BaseModel):
    """Identifying information about entities."""

    nodes: List[str] = Field(
        ...,
        description="All the Disease, Cause, Symptom, Complication, Treatment, Medication, Test, Risk Factor, Diagnosis, Side Effect, Procedure, Condition Stage, Disease Type, Imaging Type"
                    "that appear in the text"
    )

In [33]:
# Define the prompt template with instructions for node extraction
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are tasked with extracting the diseases, conditions, Diagnosis or symptoms from the qiven question "
            " for example, for the question: How is a gastric ulcer diagnosed?, you are going to find that gastric ulcer is a dieases and you also found that diagnosis is used in the question, so you are going to return gastric ulcer"
            # "You are tasked with extracting Disease, Cause, Symptom, Complication, Treatment, Medication, Test, Risk Factor, Diagnosis, Side Effect, Procedure, Condition Stage, Disease Type, Imaging Type from the text"
        ),
        (
            "human",
            "Use the specified format to extract information from the following input: {question}."
        ),
    ]
)

In [34]:
entity_chain = prompt | llm.with_structured_output(Entities)

In [35]:
entity_chain.invoke({"question": "How do NSAIDs induce gastric mucosal injury leading to ulcers?"})

Entities(nodes=['Gastric ulcer', 'NSAID-induced gastric mucosal injury'])

In [36]:
def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

In [50]:
# def structured_retriever(question: str) -> str:
#     result = ""
#     # Extract entities
#     entities = entity_chain.invoke({"question": question})
#     if not entities.nodes:
#         raise ValueError(f"No entities extracted from the question: {question}")
    
#     # Query each entity
#     for entity in entities.nodes:
#         query = generate_full_text_query(entity)
#         if not query:
#             raise ValueError(f"Generated an empty full-text query for entity: {entity}")
        
#         # Execute the full-text query
#         response = graph.query(
#             """
#             CALL db.index.fulltext.queryNodes('entityIndex', $query)
#             YIELD node, score
#             RETURN node.name AS name, score
#             """,
#             {"query": query}
#         )
        
#         # Handle no results
#         if not response:
#             result += f"No results found for entity: {entity}\n"
#         else:
#             result += "\n".join([f"Entity: {record['name']}, Score: {record['score']}" for record in response])
#     return result


In [52]:
# Fulltext index query
def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.nodes:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query)
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [54]:
print(structured_retriever("How is a gastric ulcer diagnosed?"))

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 93} for query: "CALL db.index.fulltext.queryNodes('entity', $query)\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output\n            "


Gastric ulcer - ASSOCIATED_WITH -> Bleeding
Gastric ulcer - ASSOCIATED_WITH -> Peptic ulcer disease
Gastric ulcer - ASSOCIATED_WITH -> Mass
Gastric ulcer - HAS_SYMPTOM -> Vomiting
Gastric ulcer - HAS_SYMPTOM -> Nausea
Gastric ulcer - HAS_SYMPTOM -> Pain soon after meals
Gastric ulcer - HAS_SYMPTOM -> Weight loss
Gastric ulcer - HAS_SYMPTOM -> Anorexia
H2RAs - TREATED_WITH -> Gastric ulcer
PPIs - TREATED_WITH -> Gastric ulcer
Sucralfate - TREATED_WITH -> Gastric ulcer
Misoprostol - TREATED_WITH -> Gastric ulcer
Proton pump inhibitors - TREATED_WITH -> Gastric ulcer
P-CAB - TREATED_WITH -> Gastric ulcer
Pirenzepine - TREATED_WITH -> Gastric ulcer
proton pump inhibitors - TREATED_WITH -> gastric ulcer
Gastric Ulcer - HAS_SYMPTOM -> ulcer stenosis; pyloric stenosis; oesophageal gastric varices; Barrett's oesophagus measuring >3 cm; intractable ulcer; digestive ulcer perforation or malignancy on upper GI endoscopy
Gastric Ulcer - HAS_SYMPTOM -> GI symptoms
Gastric Ulcer - COMPLICATES -> adv

In [55]:
def combined_retriever(queries):
    combined_results = []
    for query in queries:
        structured_data = structured_retriever(query)
        unstructured_data = [
            doc.page_content for doc in vector_index.similarity_search(query)
        ]
        combined_results.append({"structured": structured_data, "unstructured": "#Document ". join(unstructured_data)})
    return combined_results

In [60]:
# Combining results and generating the final response
def combined_fusion_graph_rag(original_query):
    # Step 1: Generate query variations
    raw_output = generate_queries.invoke(original_query)

    # Debugging: print raw output
    print("Raw Output from generate_queries:", raw_output)

    # Extract queries from raw output
    queries = []
    for line in raw_output:
        line = line.strip()
        if line.startswith('"') and line.endswith('",'):
            # Remove trailing comma and quotes
            queries.append(line.strip('",'))
        elif line.startswith('"') and line.endswith('"'):
            queries.append(line.strip('"'))

    # Debugging: print parsed queries
    print("Parsed Queries:", queries)
    
    # Step 2: Retrieve data for each query
    retrieved_data = combined_retriever(queries)
    
    # Step 3: Combine retrieved results
    combined_context = "\n".join(
        f"Structured:\n{result['structured']}\nUnstructured:\n{' '.join(result['unstructured'])}"
        for result in retrieved_data
    )
    
    # Step 4: Generate final response
    answer_template = """Answer the question based only on the following context:
    {context}
    
    Question: {question}
    """
    final_prompt = ChatPromptTemplate.from_template(answer_template)
    chain = final_prompt | llm | StrOutputParser()
    return chain.invoke({"context": combined_context, "question": original_query})

In [61]:
final_result = combined_fusion_graph_rag("How is a gastric ulcer diagnosed?")
print(final_result)

Raw Output from generate_queries: ['Here are 5 variations of the original question:', '', '```', '[', '    "What tests can be used to detect stomach ulcers?",', '    "Why are endoscopies commonly used in gastric ulcer diagnosis?",', '    "How do doctors figure out if someone has a stomach ulcer?",', '    "Can an upper endoscopy be used to diagnose all types of gastric ulcers, or are there other methods more effective for certain cases?",', '    "What diagnostic procedures and medical tests are typically used to confirm the presence of a gastric ulcer?"', ']', '```', '', 'These variations maintain the original intent of the question but change the phrasing, structure, or tone to create diverse queries.']
Parsed Queries: ['What tests can be used to detect stomach ulcers?', 'Why are endoscopies commonly used in gastric ulcer diagnosis?', 'How do doctors figure out if someone has a stomach ulcer?', 'Can an upper endoscopy be used to diagnose all types of gastric ulcers, or are there other 

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 93} for query: "CALL db.index.fulltext.queryNodes('entity', $query)\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.Fe

A gastric ulcer is typically diagnosed using a combination of clinical evaluation, laboratory tests, and imaging studies. Here are some common methods used to diagnose a gastric ulcer:

1. **Endoscopy**: A flexible tube with a camera and light on the end (endoscope) is inserted through the mouth to visualize the inside of the stomach and duodenum (the first part of the small intestine). The doctor can look for signs of ulcers, such as bleeding or inflammation.
2. **Biopsy**: During the endoscopy, a biopsy may be taken from the suspected ulcer site to examine tissue under a microscope for signs of inflammation or cancer.
3. **Imaging studies**: X-rays, CT scans, or MRI scans may be used to visualize the stomach and duodenum and help identify ulcers.
4. **Upper GI series**: A barium swallow test is performed by swallowing a liquid containing barium sulfate, which shows up on X-ray images. This helps visualize the stomach and duodenum and can show signs of ulcers.
5. **Endoscopic ultrasou